In [14]:
import numpy as np

'''
generate data set SYNSEP
a 9-class, 400-dimensional synthetic data set of size 10^6
'''
d = 400  # number of features
K = 9  # number of classes
n = 100  # number of data
# generate the fixed 9 bit-vectors
V = np.zeros([d, K], int)
idx_support = np.zeros([120, 9], int)
Nsupport = np.zeros([9], int)
for i in range(K):
    idx_permuted = np.random.permutation(range(120))
    idx_support[:, i] = idx_permuted
    Nsupport[i] = np.random.randint(20, 41)
    V[idx_permuted[:Nsupport[i]], i] = np.ones([Nsupport[i]])


In [15]:
# generate the data set
X = np.zeros([d, n])
for i in range(n):
    j = np.random.randint(0, 9)
    X[:, i] = V[:, j]
    idx_turnoff = np.random.permutation(idx_support[:Nsupport[j], j])[:5]
    idx_turnon = np.random.permutation(range(120, 400))[:20]
    X[idx_turnoff, i] = np.zeros([5])
    X[idx_turnon, i] = np.ones([20])

In [25]:
'''
Second Order Banditron Algorithm
'''
# initialization
a = 1  # regularization parameter
gamma = 2e-7  # exploration parameter
T = 100  # number of rounds
W1 = np.zeros([K, d])
A0 = a * np.eye(K*d)
theta0 = np.zeros([K*d])

array([0., 0., 0., ..., 0., 0., 0.])